# Create a Out of Time Dataset For Use In a Dashboard
* October 2022.


In [ ]:
import pandas as pd
import numpy as np

import datetime
from datetime import datetime

import boto3
import awswrangler
# set name of S3 bucket
s3_bucket = 'traffic-data-bucket'

import h3

import pickle


In [ ]:
from aws_secrets import aws_access_key_id, aws_secret_access_key, aws_session_token

my_session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token = aws_session_token

)

## 1. Import datasets
### 1.1 Base data


In [ ]:
base_df = awswrangler.s3.read_csv(path = f's3://{s3_bucket}/joined_data/base_location_data.csv',
                       boto3_session=my_session, use_threads=True
                       )

In [ ]:
base_df = base_df[base_df.valid_accident_location_filter == True]
base_df.shape

In [ ]:
keep_cols_list = ['hex_id', 
                  'node_street_count', 
                  'node_stop', 
                  'node_traffic_signals',
                  'CITY_NAME', 
                  'edge_speed_kph_max',
                  'edge_speek_kph_min', 
                  'edge_lanes_max', 
                  'edge_motorway_id', 
                  'edge_motorway_link_id', 
                  'edge_living_street_id',
                  'edge_bridge_id', 
                  'edge_oneway_id', 
                  'edge_tunnel_id',
                 ]

In [ ]:
base_df = base_df[keep_cols_list]

In [ ]:
base_df['geometry'] = base_df['hex_id'].apply(lambda x: h3.h3_to_geo_boundary(h=x, geo_json=True))
base_df.sample()

### 1.2. Prior Years Accident Information
Various stats regarding collision history for the hexegon and its neighbors

In [ ]:
# name of folder in S3 containing joined data
raw_path_dir = 'joined_data'

In [ ]:
prior_collision_hist_dict = {}
coll_year_dict = [2021, 2022]

for key in coll_year_dict:
    print(key)
    #base_dict[key] = pd.read_csv(root / 'X.data' / 'joined_data' / ('base_location_' + str(key) + '_collision_data.csv'))
    
    raw_path = f"s3://{s3_bucket}/{raw_path_dir}/{'base_location_' + str(key) + '_collision_data.csv'}"
    prior_collision_hist_dict[key] = awswrangler.s3.read_csv(path = raw_path, boto3_session=my_session, use_threads=True)


In [ ]:
for key in coll_year_dict:
    temp_df = prior_collision_hist_dict[key]
    temp_df['collision_year'] = key
    prior_collision_hist_dict[key] = temp_df

In [ ]:
#stack the dictionary of pd frames
prior_collision_hist_df = pd.concat(prior_collision_hist_dict.values(), ignore_index=True)

In [ ]:
prior_collision_hist_df.sample(3)

### 1.4. Weather data
Weather data for LA county.

In [ ]:
weather_df = awswrangler.s3.read_csv(path = f's3://{s3_bucket}/weather/LA_weather_data_updated.csv', boto3_session=my_session, use_threads=True)

In [ ]:
weather_df.sample(2)

### 1.5. Amenities data
Open Streets information for counts
* Restaurants, bars, colleges and schools

In [ ]:
ammenities_df = awswrangler.s3.read_csv(path = f's3://{s3_bucket}/nodes_and_edges/la_county_amenities/la_county_ammenities.csv', boto3_session=my_session, use_threads=True)

In [ ]:
ammenities_df.sample(2)

## 2. Join datasets
* base data - join on hex id
* collision history - join on hex id, collision year


In [ ]:
base_df['ttv_split'] = 'Validation_Out_Of_Time'

In [ ]:
base_df['collision_year'] = 2022

In [ ]:
joined_df1 = base_df.merge(prior_collision_hist_df, on = ['hex_id', 'collision_year'], how = 'left')

In [ ]:
#should have the same number of rows
joined_df1.shape[0] == base_df.shape[0]

In [ ]:
joined_df1

In [ ]:
for i in range(274, 274+31):
    print(i, end=" ")

#### Create a record for every hour and day in Feb and January

In [ ]:
def get_iterations(doy_start, doy_end):
    doy_period = ""
    for i in range(doy_start, doy_end):
        #print(i)
        doy_period = doy_period + " " +str(i)
    doy_period = doy_period.strip()
    return(doy_period)

In [ ]:
joined_df1['doy'] = 0
#oct 1 is the 274th day of the year.
joined_df1['doy'] = joined_df1.doy.apply(lambda x: get_iterations(274, 274+31))
joined_df1['doy'] = joined_df1['doy'].str.split()
joined_df1 = joined_df1.explode('doy')
joined_df1['doy'] = joined_df1['doy'].map(int)
joined_df1['doy'] = joined_df1['doy']
joined_df1.sample(5)

In [ ]:
joined_df1['collision_hour'] = joined_df1.doy.apply(lambda x: get_iterations(0,24))
joined_df1['collision_hour'] = joined_df1['collision_hour'].str.split()
joined_df1 = joined_df1.explode('collision_hour')
joined_df1['collision_hour'] = joined_df1['collision_hour'].map(str)
joined_df1.sample(5)

In [ ]:
joined_df1.collision_hour.unique()

In [ ]:
joined_df1.doy.unique()

In [ ]:
joined_df1['collision_hour_grp'] = '1.6AM_to_9AM;Morning Commute'
joined_df1.loc[joined_df1['collision_hour'].isin(['10', '11', '12', '13']), 'collision_hour_grp'] = '2.10AM_to_1PM;Late Morning'
joined_df1.loc[joined_df1['collision_hour'].isin(['14', '15', '16', '17']), 'collision_hour_grp'] = '3.2PM_to_5PM;Afternoon'
joined_df1.loc[joined_df1['collision_hour'].isin(['18', '19', '20', '21']), 'collision_hour_grp'] = '4.6PM_to_9PM;Evening Commute'
joined_df1.loc[joined_df1['collision_hour'].isin(['22', '23', '0', '1']), 'collision_hour_grp'] = '5.10PM_to_1AM;Night'
joined_df1.loc[joined_df1['collision_hour'].isin(['2', '3', '4', '5']), 'collision_hour_grp'] = '6.2AM_to_5AM;Early Morning'

In [ ]:
#check. 
joined_temp =joined_df1.copy()
joined_temp.collision_hour = joined_temp.collision_hour.map(int)
#check the output to make sure
display(joined_temp.groupby(['collision_hour', 'collision_hour_grp']).size().reset_index().sort_values('collision_hour'))
del joined_temp

In [ ]:
joined_df1['Yd'] = joined_df1['collision_year'].map(str) + joined_df1['doy'].map(str)

In [ ]:
joined_df1['collision_date'] = pd.to_datetime(joined_df1['Yd'].map(int), format='%Y%j')

In [ ]:
joined_df1['collision_dayofweek'] = joined_df1['collision_date'].dt.dayofweek
joined_df1['collision_month'] = joined_df1['collision_date'].dt.month
joined_df1['collision_dayofmonth'] = joined_df1['collision_date'].dt.day

In [ ]:
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.sample(5)

In [ ]:
weather_df['month'] = weather_df['date'].dt.month
weather_df['dayofmonth'] = weather_df['date'].dt.day
weather_df_grp = weather_df.groupby(['month', 'dayofmonth'])['awnd','prcp','tavg','tmax','tmin','wdf2'].mean()
weather_df_grp = weather_df_grp.reset_index()

In [ ]:
#make sure they are the same format
joined_df1['collision_day'] = pd.to_datetime(joined_df1['collision_date']).dt.day
joined_df2 = joined_df1.merge(weather_df_grp[weather_df_grp.month == 10], 
                              left_on = ['collision_month', 'collision_day'], 
                              right_on = ['month', 'dayofmonth'], how = 'left')

In [ ]:
with pd.option_context("display.max_columns", 150):
    display(joined_df2.sample(5))

In [ ]:
#should have the same number of rows
joined_df2.shape[0] == joined_df1.shape[0]

In [ ]:
joined_df3 = joined_df2.merge(ammenities_df, on = ['hex_id'], how = 'left')

In [ ]:
#should have the same number of rows
joined_df2.shape[0] == joined_df3.shape[0]

## 3 Clean up the columns to reflect model script

In [ ]:
joined_df3.prev1_yr_coll_cnt = joined_df3.prev1_yr_coll_cnt.fillna(0)
joined_df3.prev2_yr_coll_cnt = joined_df3.prev2_yr_coll_cnt.fillna(0)
joined_df3.prev1_yr_coll_neighbor1 = joined_df3.prev1_yr_coll_neighbor1.fillna(0)
joined_df3.prev1_yr_coll_neighbor2 = joined_df3.prev1_yr_coll_neighbor2.fillna(0)
joined_df3.prev2_yr_coll_neighbor1 = joined_df3.prev2_yr_coll_neighbor1.fillna(0)
joined_df3.prev2_yr_coll_neighbor2 = joined_df3.prev2_yr_coll_neighbor2.fillna(0)
joined_df3.amenities_restaurant_cnt = joined_df3.amenities_restaurant_cnt.fillna(0)
joined_df3.amenities_bar_cnt = joined_df3.amenities_bar_cnt.fillna(0)
joined_df3.amenities_school_cnt = joined_df3.amenities_school_cnt.fillna(0)
joined_df3.amenities_college_cnt = joined_df3.amenities_college_cnt.fillna(0)


In [ ]:
joined_df3.rename(columns = {'awnd':'noaa_wind_speed' 
                                    ,'prcp':'noaa_precipitation'
                                    ,'tavg':'noaa_temperature_average'
                                    ,'tmax':'noaa_temperature_max'
                                    ,'tmin':'noaa_temperature_min'
                                    ,'wdf2':'noaa_wind_direction'
                                    ,'CITY_NAME':'la_data_city_name'
                                    ,'edge_motorway_id':'edge_motorway_flag' 
                                    ,'edge_motorway_link_id':'edge_motorway_link_flag'
                                    ,'edge_living_street_id':'edge_living_street_flag'
                                    ,'edge_bridge_id':'edge_bridge_flag' 
                                    ,'edge_oneway_id':'edge_oneway_flag'
                                    ,'edge_tunnel_id':'edge_tunnel_flag'
                                    ,'drv_holiday_id':'drv_holiday_flag'
                                    ,'accident_count':'target'
                                   }, inplace = True)

In [ ]:
max_edge_mode = joined_df3.edge_lanes_max.value_counts().index[0]
max_edge_mode

In [ ]:
joined_df3['drv_edge_lanes_max_imputed_flag'] = joined_df3['edge_lanes_max'].isna()

In [ ]:
joined_df3['drv_edge_lanes_max_imputed_flag'].value_counts()

In [ ]:
joined_df3.edge_lanes_max = joined_df3.edge_lanes_max.fillna(max_edge_mode)

### Make a holiday indicator

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import datetime

cal = calendar()
dr = pd.date_range(start='2014-01-01', end='2020-12-31')
df = pd.DataFrame()
df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

holiday_list = list()

for holiday in holidays:
    
    holiday = holiday.date()

    date_plus1 = holiday + datetime.timedelta(days=1)
    date_minus1 = holiday + datetime.timedelta(days=-1)
    
    holiday_list.append(date_minus1.strftime('%Y-%m-%d'))
    holiday_list.append(holiday.strftime('%Y-%m-%d'))
    holiday_list.append(date_plus1.strftime('%Y-%m-%d'))
    holiday_list.append(date_minus1.strftime('%Y-%m-%d'))
    
holiday_list[0:10]


In [ ]:
joined_df3['collision_date'] = joined_df3['collision_date'].map(str)

In [ ]:
joined_df3['drv_holiday_flag'] = joined_df3['collision_date'].isin(holiday_list)

In [ ]:
#update true false id columns
for col in joined_df3.columns:
    if (col.endswith('_flag')):
        joined_df3.loc[joined_df3[col] == True, col] = 1
        joined_df3.loc[joined_df3[col] == False, col] = 0
        joined_df3[col] = joined_df3[col].astype('int')

In [ ]:
joined_df3['drv_holiday_flag'].value_counts()

### Transform Time Component

In [ ]:
def cyclical_encode(data, col, max_val):
    data['drv_' + col + '_sin'] = np.sin(2 * np.pi * data[col].map(int)/max_val)
    data['drv_' + col + '_cos'] = np.cos(2 * np.pi * data[col].map(int)/max_val)
    return data

In [ ]:
sin_cos_transformed_list = {'collision_hour' : 23 ,
                            'collision_dayofweek' : 6, 
                            'collision_month' : 12
                           }

for date_type in sin_cos_transformed_list:
    joined_df3 = cyclical_encode(joined_df3, date_type, sin_cos_transformed_list[date_type])

### Import collision to create target

In [ ]:
joined_df3.shape

In [ ]:
with pd.option_context("display.max_columns", 150):
    display(joined_df3.sample(5))

In [ ]:
max(joined_df3.prev1_yr_coll_neighbor1)

In [ ]:
# fix double counting of history
joined_df3['history_check'] = joined_df3.prev1_yr_coll_neighbor1 - joined_df3.prev1_yr_coll_cnt

In [ ]:
joined_df3['history_check'].min()

## 4. Save final dataset
#### Write to parquet and upload to S3

In [ ]:
awswrangler.s3.to_parquet(df=joined_df3, path = f's3://{s3_bucket}/model_data/out_of_time_validation.parquet', index=False,
                       boto3_session=my_session, use_threads=True
                       )